# Neural Network Model


In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
import pandas as pd

# # Import and read the csv.
# b2 view
df = pd.read_csv("../ML_Data_&_Preprocessing/b2_df_nonEncoded.csv", index_col=[0])
df.head()



,state,spend,Impressions,leads,link_clicks,reach,Agency Tiers
0,IL,47.33,1780,0,7,1689,4.0
1,NY,63.65,1857,1,10,1737,4.0
2,OK,32.53,1718,0,7,1527,4.0
3,SC,32.31,1725,1,14,1645,4.0
4,CA,101.13,3745,1,28,3513,4.0


In [2]:
df['leads'].value_counts()

1    21698
0    12272
Name: leads, dtype: int64

In [3]:
# check on different state values for testing 
df['state'].value_counts()

CA    7685
TX    4254
VA    3239
CO    2386
FL    1758
MI    1655
GA    1620
NY    1486
NC    1050
TN     970
WA     960
NJ     919
CT     893
OH     842
MO     777
MN     628
IL     605
MA     454
SC     365
OK     365
MD     357
DC     250
SD     246
NE     173
PA      33
Name: state, dtype: int64

In [4]:
df2 = df.loc[df['state']== 'FL']
df2

,state,spend,Impressions,leads,link_clicks,reach,Agency Tiers
109,FL,15.67,852,1,18,815,4.0
189,FL,16.21,855,0,28,814,4.0
231,FL,46.14,1489,1,17,1342,4.0
314,FL,15.85,798,1,16,733,4.0
330,FL,16.37,1208,1,25,1159,4.0
...,...,...,...,...,...,...,...
33077,FL,52.45,2685,1,30,2514,NaN
33118,FL,52.09,2662,1,23,2511,NaN
33146,FL,44.56,2389,1,28,2239,NaN
33147,FL,41.74,2529,1,35,2342,NaN


In [5]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1758 entries, 109 to 33196
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   state         1758 non-null   object 
 1   spend         1758 non-null   float64
 2   Impressions   1758 non-null   int64  
 3   leads         1758 non-null   int64  
 4   link_clicks   1758 non-null   int64  
 5   reach         1758 non-null   int64  
 6   Agency Tiers  668 non-null    float64
dtypes: float64(2), int64(4), object(1)
memory usage: 109.9+ KB


In [6]:
df3 = df2.drop(df2.columns[[0,6]], axis=1)

In [7]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1758 entries, 109 to 33196
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   spend        1758 non-null   float64
 1   Impressions  1758 non-null   int64  
 2   leads        1758 non-null   int64  
 3   link_clicks  1758 non-null   int64  
 4   reach        1758 non-null   int64  
dtypes: float64(1), int64(4)
memory usage: 82.4 KB


# Split into train and test features

In [8]:
# Split our preprocessed data into our features and target arrays
y = df3['leads'].values
X = df3.drop(['leads'], 1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

C:\Users\dia78039\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


In [9]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Compile, Train, Evaluate our Model 1 - NN

In [10]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
# using multi-layer perceptron (two layers)
numInputFeatures = len(X_train[0])

# I have X amount of columns
# layer1 = input layer, typically equals number of input variables in data
layer1 = 30
# layer 2 = hidden layer, typically 2/3 of input layer
layer2 = 15
# layer 3 = hidden layer
layer3= 2

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=layer1, 
                          input_dim=numInputFeatures, 
                          activation="hard_sigmoid")
)
# Second hidden layer
nn.add(tf.keras.layers.Dense(units=layer2, 
                             activation="elu"))

# adding a third layer to increase accuracy 
nn.add(tf.keras.layers.Dense(units=layer3, 
                             activation='elu'))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="hard_sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 30)                150       
                                                                 
 dense_1 (Dense)             (None, 15)                465       
                                                                 
 dense_2 (Dense)             (None, 2)                 32        
                                                                 
 dense_3 (Dense)             (None, 1)                 3         
                                                                 
Total params: 650
Trainable params: 650
Non-trainable params: 0
_________________________________________________________________


In [11]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    # checkpoint directory and file structure defined above
    filepath=checkpoint_path,
    # notified when checkpoint is being saved to the directory
    verbose=1,
    # checkpoint files take small space
    save_weights_only=True,
    # checkpoints saved every epoch
    save_freq='epoch')

In [12]:
# Train the model
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])

Epoch 1/100
30/42 [====================>.........] - ETA: 0s - loss: 0.5554 - accuracy: 0.7604
Epoch 1: saving model to checkpoints\weights.01.hdf5
42/42 [==============================] - 1s 5ms/step - loss: 0.5502 - accuracy: 0.7557
Epoch 2/100
39/42 [==========================>...] - ETA: 0s - loss: 0.4760 - accuracy: 0.7821
Epoch 2: saving model to checkpoints\weights.02.hdf5
42/42 [==============================] - 0s 5ms/step - loss: 0.4738 - accuracy: 0.7838
Epoch 3/100
33/42 [======================>.......] - ETA: 0s - loss: 0.4260 - accuracy: 0.8277
Epoch 3: saving model to checkpoints\weights.03.hdf5
42/42 [==============================] - 0s 4ms/step - loss: 0.4234 - accuracy: 0.8331
Epoch 4/100
40/42 [===========================>..] - ETA: 0s - loss: 0.3824 - accuracy: 0.8414
Epoch 4: saving model to checkpoints\weights.04.hdf5
42/42 [==============================] - 0s 4ms/step - loss: 0.3776 - accuracy: 0.8460
Epoch 5/100
31/42 [=====================>........] - ETA: 0s

Epoch 36/100
40/42 [===========================>..] - ETA: 0s - loss: 0.2962 - accuracy: 0.8680
Epoch 36: saving model to checkpoints\weights.36.hdf5
42/42 [==============================] - 0s 3ms/step - loss: 0.2960 - accuracy: 0.8687
Epoch 37/100
39/42 [==========================>...] - ETA: 0s - loss: 0.2963 - accuracy: 0.8718
Epoch 37: saving model to checkpoints\weights.37.hdf5
42/42 [==============================] - 0s 4ms/step - loss: 0.2948 - accuracy: 0.8725
Epoch 38/100
30/42 [====================>.........] - ETA: 0s - loss: 0.2830 - accuracy: 0.8740
Epoch 38: saving model to checkpoints\weights.38.hdf5
42/42 [==============================] - 0s 4ms/step - loss: 0.2971 - accuracy: 0.8680
Epoch 39/100
35/42 [========================>.....] - ETA: 0s - loss: 0.2883 - accuracy: 0.8696
Epoch 39: saving model to checkpoints\weights.39.hdf5
42/42 [==============================] - 0s 4ms/step - loss: 0.2943 - accuracy: 0.8703
Epoch 40/100
31/42 [=====================>........] 

42/42 [==============================] - 0s 2ms/step - loss: 0.2924 - accuracy: 0.8710
Epoch 71/100
31/42 [=====================>........] - ETA: 0s - loss: 0.2894 - accuracy: 0.8720
Epoch 71: saving model to checkpoints\weights.71.hdf5
42/42 [==============================] - 0s 3ms/step - loss: 0.2925 - accuracy: 0.8665
Epoch 72/100
41/42 [============================>.] - ETA: 0s - loss: 0.2958 - accuracy: 0.8704
Epoch 72: saving model to checkpoints\weights.72.hdf5
42/42 [==============================] - 0s 3ms/step - loss: 0.2949 - accuracy: 0.8710
Epoch 73/100
39/42 [==========================>...] - ETA: 0s - loss: 0.2911 - accuracy: 0.8694
Epoch 73: saving model to checkpoints\weights.73.hdf5
42/42 [==============================] - 0s 3ms/step - loss: 0.2927 - accuracy: 0.8687
Epoch 74/100
42/42 [==============================] - ETA: 0s - loss: 0.2932 - accuracy: 0.8695
Epoch 74: saving model to checkpoints\weights.74.hdf5
42/42 [==============================] - 0s 3ms/step

In [13]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

# when brand 1, all states, accuracy = 36%
# when brand 2, all states, accuracy = 36%
# when brand 3, all states, accuracy = 36%

# FL = .84%
# 


14/14 - 0s - loss: 0.3376 - accuracy: 0.8409 - 164ms/epoch - 12ms/step
Loss: 0.3376096189022064, Accuracy: 0.8409090638160706


### B2 Results

#### B2

In [16]:
data_results_b2 = {
        'State': ['CA', 'FL'],
        'Predictive Accuracy': [.80, .84],
        'Notes': ['N/A', 'N/A']}
data_results_b2

{'State': ['CA', 'FL'],
 'Predictive Accuracy': [0.8, 0.84],
 'Notes': ['N/A', 'N/A']}

In [17]:
# create the new df to display brand, state, and model accuracy 
df_results_b2 = pd.DataFrame(data_results_b2)
df_results_b2

,State,Predictive Accuracy,Notes
0,CA,0.80,N/A
1,FL,0.84,N/A


In [ ]:
# # Export the model to HDF5 file
# nn.save("AlphabetSoupCharity_optimization.h5")